# Pydantic POC 

In [1]:
import numpy as np
import pandas as pd

## Example 

In [1]:
from datetime import date
from typing import Tuple

from pydantic import BaseModel, ConfigDict, ValidationError


class Event(BaseModel):
    model_config = ConfigDict(strict=True)

    when: date
    where: Tuple[int, int]

In [2]:
json_data = '{"when": "1987-01-28", "where": [51, -1]}'
print(Event.model_validate_json(json_data))

when=datetime.date(1987, 1, 28) where=(51, -1)


In [3]:
try:
    Event.model_validate({'when': '1987-01-28', 'where': [51, -1]})  
except ValidationError as e:
    print(e)

2 validation errors for Event
when
  Input should be a valid date [type=date_type, input_value='1987-01-28', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/date_type
where
  Input should be a valid tuple [type=tuple_type, input_value=[51, -1], input_type=list]
    For further information visit https://errors.pydantic.dev/2.7/v/tuple_type


## Try types

In [8]:
class Type1(BaseModel):

    expression_type: str
    value: int

In [10]:
type1_data = """\
    {"expression_type": "Type1", "value": 7}
"""
Type1.model_validate_json(type1_data)

Type1(expression_type='Type1', value=7)

In [11]:
class Type2(BaseModel):

    expression_type: str
    value1: int
    value2: float

In [27]:
type2_data = """\
    {"expression_type": "Type2", "value1": 7, "value2": 7.7}
"""
Type2.model_validate_json(type2_data)

Type2(expression_type='Type2', value1=7, value2=7.7)

In [40]:
unsupported_type_data = """\
    {"expression_type": "foo", "value1": 7, "value2": 7.7}
"""
Type2.model_validate_json(unsupported_type_data)

Type2(expression_type='foo', value1=7, value2=7.7)

In [36]:
from pydantic_core import from_json

parsed_dict = from_json(type1_data, allow_partial=True)
parsed_dict

{'expression_type': 'Type1', 'value': 7}

In [37]:
from enum import Enum

class ExpressionParser(Enum):
    Type1: BaseModel = Type1
    Type2: BaseModel = Type2

    @classmethod
    def parse(cls, json_data):
        expr_type = from_json(json_data, allow_partial=True)['expression_type']
        return cls[expr_type].value.model_validate_json(json_data)

ExpressionRouter.parse(type1_data)

Type1(expression_type='Type1', value=7)

In [38]:
ExpressionRouter.parse(type2_data)

Type2(expression_type='Type2', value1=7, value2=7.7)

In [41]:
ExpressionRouter.parse(unsupported_type_data)

KeyError: 'foo'